In [1]:
import pandas as pd
import numpy as np
# import stanfordnlp
import time
import scispacy
from tqdm import tqdm
import itertools
import pickle
import sys
import re, nltk
import os
from heuristic_tokenize import sent_tokenize_rules

In [2]:
#setting sentence boundaries
def sbd_component(doc):
    for i, token in enumerate(doc[:-2]):
        # define sentence start if period + titlecase token
        if token.text == '.' and doc[i+1].is_title:
            doc[i+1].sent_start = True
        if token.text == '-' and doc[i+1].text != '-':
            doc[i+1].sent_start = True
    return doc

#convert de-identification text into one token
def fix_deid_tokens(text, processed_text):
    deid_regex  = r"\[\*\*.{0,15}.*?\*\*\]" 
    if text:
        #print("!!!!!")
        indexes = [m.span() for m in re.finditer(deid_regex,text,flags=re.IGNORECASE)]
    else:
        #print("??????")
        indexes = []
    #print(indexes)
    for start,end in indexes:
        a = processed_text.merge(start_idx=start,end_idx=end)
        #print(a)
    #print(text)
    return processed_text
    

def process_section(section, note, processed_sections):
    # perform spacy processing on section
    #print("--------- before ---------")
    #print(section['sections'])
    processed_section = nlp(section['sections'])
    #print("--------- AFTER ----------")
    #print(processed_section)
    processed_section = fix_deid_tokens(section['sections'], processed_section)
    processed_sections.append(processed_section)

def process_note_helper(note):
    # split note into sections
    note_sections = sent_tokenize_rules(note)
    processed_sections = []
    section_frame = pd.DataFrame({'sections':note_sections})
    
    section_frame.apply(process_section, args=(note,processed_sections,), axis=1)
    return(processed_sections)

def process_text(sent, note):
    sent_text = sent['sents'].text
    sent_text = sent_text.replace("_","")
    sent_text = sent_text.replace("#","")
    sent_text = re.sub(r"(1[0-2]|[1-9]):[0-5][0-9] *(a|p|A|P)(m|M) *", "",sent_text)
    if len(sent_text) > 0 and sent_text.strip() != '\n':
        if '\n' in sent_text:
            sent_text = sent_text.replace('\n', ' ')
#         print('sent_text:', sent_text)
        if sent_text[-1] ==':':
            note['HEADERS'] += sent_text[:-1] + ','
#         else:
        note['TEXT'] += sent_text + '\n'
    
    
    

def get_sentences(processed_section, note):
    # get sentences from spacy processing
    sent_frame = pd.DataFrame({'sents': list(processed_section['sections'].sents)})
    sent_frame.apply(process_text, args=(note,), axis=1)


def process_note(note):
    try:
        note_text = note['TEXT'] #unicode(note['text'])
        note['TEXT'] = ''
        processed_sections = process_note_helper(note_text)
        ps = {'sections': processed_sections}
        ps = pd.DataFrame(ps)
        ps.apply(get_sentences, args=(note,), axis=1)
        return note 
    except Exception as e:
        print('!!!!! error', e)
        pass 

# **en_core_sci_md:-**
A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.	

In [3]:
# !pip install spacy
import spacy
!python -m spacy download en_core_sci_md

AttributeError: partially initialized module 'torch' has no attribute 'Tensor' (most likely due to a circular import)

In [ ]:
pip uninstall spacy

In [27]:
# Load the en_core_sci_md model
nlp = spacy.load('en_core_sci_md')
print("Model loaded successfully!")

Model loaded successfully!


In [14]:
import en_core_sci_md
nlp = en_core_sci_md.load()
nlp = spacy.load('en_core_sci_md', disable=['tagger','ner'])
nlp.add_pipe(sbd_component, before='parser')

ModuleNotFoundError: No module named 'en_core_sci_md'

In [29]:
SAMPLE_20K = pd.read_csv(r'D:\FINALYEARPROJECTREC\artifacts\SAMPLE_20K.csv')

In [30]:
SAMPLE_20K['TEXT'] = SAMPLE_20K['TEXT'].replace(to_replace=r"\[.*?\]", value="", regex=True)

In [31]:
# SAMPLE_20K = SAMPLE_20K.drop(columns=['Unnamed: 0'])

In [32]:
SAMPLE_20K['HEADERS'] = ""

In [ ]:
print('Number of notes: %d' %len(SAMPLE_20K.index))
SAMPLE_20K['ind'] = list(range(len(SAMPLE_20K.index)))

In [34]:
tqdm.pandas()

In [ ]:
SAMPLE_20K = SAMPLE_20K.progress_apply(process_note, axis=1)

In [36]:
keep = SAMPLE_20K.dropna()

In [ ]:
keep

In [ ]:
discard_lst = {'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
               'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
               ' MD Phone',' M.D. Phone', ' MD',' PHD',
               ' X', ' IV', ' VI', 'III', 'II', 'VIII',
               'JOB#','JOB#: cc', '# Code',
               'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
              }
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

In [ ]:
# get top 10 sentences and get 25 vocabularies from those sentences
all_tokens = []
num_vocabs = 25
num_sentences = 10
discard_tokens = ('Sex: F', 'Sex: M', 'MEDICINE', 'Neurosurgery', 'Service: CT', 'Allergies')
for index, row in keep.iterrows():
    cur = [ ["<PAD>"] * num_vocabs for i in range(num_sentences) ]
    sents = row["TEXT"].splitlines()
    headers = ' '.join(row['HEADERS']).split()
        
    """
    for j in range(len(headers)):
        if j < num_vocabs:
            cur[0][j] = headers[j]       
    """          
    
    sentence_idx = 0
    for i in range(0, len(sents)):
        if sentence_idx >= num_sentences:
            break
        
        if sents[i].startswith(discard_tokens):
            continue
        
        vocabs = sents[i].split()
        
        for j in range(len(vocabs)):  
            if(j >= num_vocabs):
                break
            
            #print(str(sentence_idx)+" "+str(j)+" "+vocabs[j])
            cur[sentence_idx][j] = vocabs[j]
        
        sentence_idx += 1
    
    all_tokens.append(list(itertools.chain.from_iterable(cur)))
keep['padded_tokens'] = all_tokens

In [ ]:
keep

In [ ]:
keep.to_pickle("./abhi.pkl")
keep.to_csv('abhi.csv',index=True)

In [ ]:
keep = pd.read_csv("abhi.csv")

In [ ]:
keep = keep.drop(columns=['Unnamed: 0'])

In [ ]:
keep2 = SAMPLE_20K.dropna()

In [ ]:
def clean(text):
    output=re.sub(r'\n',' ',text)
    output = re.sub(r" +", " ",output)
    output = output.strip()
    return output

In [ ]:
keep['CLEANED TEXT']=keep['TEXT'].apply(clean)


In [ ]:
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 
    
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [ ]:
discard_list={'','Admission Date', 'Discharge Date','Date of Birth', 'Phone','Date/Time', 'ID',
            'Completed by', 'Dictated By','Attending', 'Provider: ','Provider', 'Primary', 'Secondary', 
            ' MD Phone',' M.D. Phone', ' MD',' PHD',
            ' X', ' IV', ' VI', 'III', 'II', 'VIII',
            'JOB#','JOB#: cc', '# Code',
            'Metoprolol Tartrate 25 mg Tablet Sig',')','000 unit/mL Suspension Sig',' ','0.5 % Drops ','   Status: Inpatient DOB', 'Levothyroxine 50 mcg Tablet Sig', '0.5 % Drops Sig','Lidocaine 5 %(700 mg/patch) Adhesive Patch','Clopidogrel Bisulfate 75 mg Tablet Sig','Levofloxacin 500 mg Tablet Sig','Albuterol 90 mcg/Actuation Aerosol ','None Tech Quality: Adequate Tape #','000 unit/mL Solution Sig',
            }
# keep['HEADERS'] = keep['HEADERS'].apply(lambda x: x.split(','))
keep['HEADERS'] = keep['HEADERS'].apply(lambda x: list(set(x).difference(discard_lst)))

In [ ]:
keep['CLEAN_WORDS']=keep['TEXT'].apply(preprocess)

In [ ]:
from nltk.stem import *
stemmer=PorterStemmer()

In [ ]:
keep['STEM_WORDS']=keep['CLEAN_WORDS'].apply(lambda x: [stemmer.stem(w) for w in x])

In [ ]:
corpus={}
for i,s in enumerate(keep['STEM_WORDS']):
    for w in s:
        corpus[w]=corpus.get(w,1)+1
corpus={k: v for k,v in sorted(corpus.items(),key=lambda item: item[1],reverse=True)}


In [ ]:
len(corpus)

In [ ]:
corpus_slice=dict(itertools.islice(corpus.items(),10000))

In [ ]:
len(corpus_slice)

In [ ]:
# build dictionary
word2idx = {'<PAD>': 0, '<UNK>':1}
idx2word = {0: '<PAD>', 1:'<UNK>'}
for c in corpus_slice:
    word2idx[c] = len(word2idx)
    idx2word[len(idx2word)] = c

In [ ]:
length = 256
def note2idx_cap(org_lst):
    coded_lst = []
    for w in org_lst:
        if len(coded_lst) < length and w in word2idx:
            coded_lst.append(word2idx[w])
        else:
            coded_lst.append(1)
    coded_lst += [0]*(length-len(coded_lst))
    return coded_lst

In [ ]:
keep['CODED_TEXT']=kepp